<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/141-Llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LlamaIndex <> Langchain Integrations

This demo notebook shows how you can provide integrations between LlamaIndex and Langchain. It provides the following examples:
- Using LlamaIndex as a callable tool with a Langchain agent
- Using LlamaIndex as a memory module; this allows you to insert arbitrary amounts of conversation history with a Langchain chatbot!

In [ ]:
!pip install --upgrade openai

In [ ]:
!pip install llama-index pypdf

In [ ]:
!pip install langchain

In [ ]:
import openai

# Set up the OpenAI API client
openai.api_key = "sk-"


In [ ]:
import os

openai_api_key = os.getenv('OPENAI_API_KEY', 'sk-')

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Using LlamaIndex as a Callable Tool

In [ ]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
!mkdir data

In [ ]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents=documents)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

In [ ]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0, openai_api_key='sk-')
agent_executor = initialize_agent(
    tools, llm, agent="conversational-react-description", memory=memory
)

In [ ]:
agent_executor.run(input="hi, i am bob")

'Hello Bob! How can I assist you today?'

In [ ]:
agent_executor.run(input="What did the author do growing up?")

"The author's childhood involved working on writing and programming outside of school. They wrote short stories and tried writing programs on an IBM 1401 computer in 9th grade. They used an early version of Fortran and had to type programs on punch cards. The author also mentioned their fascination with microcomputers and eventually convinced their father to buy a TRS-80 computer. They wrote simple games and a word processor on it. Additionally, the author mentioned their initial plan to study philosophy in college but eventually switched to AI."

#### Using LlamaIndex as a memory module

In [ ]:
# try using SummaryIndex!
from langchain.llms import OpenAI
from langchain.llms import OpenAIChat
from langchain.agents import initialize_agent

from llama_index import SummaryIndex
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

In [ ]:
index = SummaryIndex([])

In [ ]:
# set Logging to DEBUG for more detailed outputs
# NOTE: you can also use a conversational chain

memory = GPTIndexChatMemory(
    index=index,
    memory_key="chat_history",
    query_kwargs={"response_mode": "compact"},
    # return_source returns source nodes instead of querying index
    return_source=True,
    # return_messages returns context in message format
    return_messages=True,
)
# llm = OpenAIChat(temperature=0)
llm=OpenAI(temperature=0, openai_api_key='sk-')
agent_executor = initialize_agent(
    [], llm, agent="conversational-react-description", memory=memory
)

In [ ]:
agent_executor.run(input="hi, i am bob")

'Hi Bob, nice to meet you! How can I help you today?'

In [ ]:
# NOTE: the query now calls the SummaryIndex memory module.
agent_executor.run(input="what's my name?")

'What is your name?'